### 事前準備

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp "/content/drive/My Drive/Tokyo_Power_weather_2017-2018_sample_new2.csv" "./"   # データのコピー
!cp "/content/drive/My Drive/Tokyo_Power_weather_Predict_sample_new2.csv" "./"
!cp "/content/drive/My Drive/Tokyo_Power_20190401_05.csv" "./"
!ls -l

# **scikit-learn(sklearn)で東京電力の電力需要を重回帰予測する例**

### 本例で用いる学習データの所在

*   **東京電力「過去の電力使用実績のダウンロード」**[ここから入手](http://www.tepco.co.jp/forecast/html/download-j.html)
*   **気象庁「過去の気象データ・ダウンロード」**[ここから入手](https://www.data.jma.go.jp/gmd/risk/obsdl/index.php)


## **データ準備**

**学習データの準備**

<font color=red>**本例では2017/1/1～2018/12/31の東京電力_電力実績と気象庁データ(気温/降水量/日照時間/風速など)の結合データ(CSV)を使用**</font>

　データフレーム名　**train_df** ：Tokyo_Power_Weather_2017-2018_sample_new2.csvをPandasで格納<br>
　※Pandas：Pythonで表形式のデータを扱うためのライブラリ。データ抽出/加工/集計など、分析でよく使われる。

In [0]:
import pandas as pd #pandasをpdとしてインポート。データをdfに読み込み。pandasをpdとして利用。
train_df = pd.read_csv("Tokyo_Power_weather_2017-2018_sample_new2.csv", encoding="utf_8")
train_df.head(5)    #データフレームの表示（先頭5行表示）

＜参考＞データサンプルのカラム説明（レコード数は17,506） \\
**DATE**:年月日(東電)、**TIME_hh**:時間(東電)、**TIME_mm**:分(東電)、<font color=red>**Man_kW**:電力量　万キロワット(東電)**←今回の目的変数**</font>、 \\
**Temp_C**:気温　℃(気象庁)、**Sun_h**:日照時間　時間(気象庁)、**Wind_m_par_S**:風速　m/秒(気象庁)、**Rain_mm**:降水量　mm(気象庁)


**データの準備** \\
説明変数となるデータ（Ｘ）を準備する。 \\
データフレームから、説明変数で不要なカラム（今回はDATEとTIME_mm）を削除する。   \\
あわせて、目的変数となるカラム（今回はMan_kW）も削除する。

In [0]:
train_df = train_df.drop("DATE",axis=1)     # 説明変数の不要カラム（DATE）を削除
train_df = train_df.drop("TIME_mm",axis=1)  # 説明変数の不要カラム（TIME_mm）を削除
train_df = train_df.drop("Man_kW",axis=1)   # 目的変数のカラム（Man_kW）を削除
train_df.head(5)    #不要カラム削除後のデータフレーム表示（先頭5行表示）

**説明変数（Ｘ）をセット** \\
上記のデータフレームを入力として、説明変数（Ｘ）をセットする。 \\
ついでに、基本統計量（即ち、Xの概略）も表示する。

In [0]:
X = train_df   # 説明変数Ｘの準備
print(X.describe())   # Ｘの概要（基本統計量）

＜補足＞</br>
describe関数で数値の特徴（基本統計量）を見ることができる。主な項目は次の通り。</br>
<ul>
<li>count：データの個数</li>
<li>mean：平均値</li>
<li>std：標準偏差（データのばらつき）</li>
<li>min/25%/50%/75%/max：四分位(しぶんい)数　全データをソートした後の代表的な数値を抽出した値。より詳細にデータのバラツキが分かる。</li>
</ul>


**説明変数（Ｘ）の確認**

In [0]:
X.head(5)  # Ｘの表示（先頭5行）

**目的変数（Ｙ）の準備と確認** \\
不要なカラムを削除して、目的変数（今回は電力量Man_kW）をセット。

In [0]:
train_df = pd.read_csv("Tokyo_Power_weather_2017-2018_sample_new2.csv", encoding="utf_8")
Y = train_df    # 元のtrain.dfからMan_kW以外を削除
Y = Y.drop("DATE",axis=1)
Y = Y.drop("TIME_hh",axis=1)
Y = Y.drop("TIME_mm",axis=1)
Y = Y.drop("Temp_C",axis=1)
Y = Y.drop("Sun_h",axis=1)
Y = Y.drop("Wind_m_par_S",axis=1)
Y = Y.drop("Rain_mm",axis=1)
Y.head(5)   # Ｙの表示（先頭5行）

ここまでで、学習データ（テスト含む）となる、Ｘ(説明変数)とＹ(目的変数)の準備が完了。

## **重回帰モデルの作成**

In [0]:
import pandas as pd    # 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split   # 学習とテストのデータを分割する時に必要
from sklearn import datasets
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# 学習用とテスト用のデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, random_state=0)   # 学習用80％、テスト用20％に分割

# 重回帰分析（予測モデルの作成）
clf = linear_model.LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=1)
clf.fit(X_train, Y_train)

#print(pd.DataFrame({"Name":X.columns,"Coefficients":np.abs(clf.coef_)}).sort_values(by='Coefficients'))
print("回帰変数：",clf.coef_)    # coef　：回帰変数
print("切片：",clf.intercept_)    # intercept_ ：切片
#print("決定係数(学習用)：",clf.score(X_train,Y_train))   # score ：決定係数 R2を出力
#print("決定係数(テスト用)：",clf.score(X_test,Y_test))   # score ：決定係数 R2を出力

**重回帰モデルの作成完了** \\
結果を式で表すと \\
Man_kW =  (41.68×TIME_hh) + (-4.81×Temp_C) + (440.17×Sun_h) + (36.94×Wind_m_par_S) +(-11.17×Rain_mm) +2694.84  \\
となる。

## **予測（Prediction）**

**予測データの準備** \\

<font color=red>**本例では2019/4/1～4/5の東京電力_電力実績と気象庁データ(気温/降水量/日照時間/風速など)の結合データ(CSV)を、将来データとして使用している。**</font>

　データフレーム名　**pred_df** ：Tokyo_Power_weather_Predict_sample_new2.csvをPandasで格納

**データのロード**

In [0]:
pred_df = pd.read_csv("Tokyo_Power_weather_Predict_sample_new2.csv", encoding="utf_8")   # 予測用データの準備
pred_df.head(5)    #  データフレームの表示（先頭5行）

**予測データの準備** \\
データフレームから不要なカラム（今回はDATEとTIME_mm）を削除する。確認のために、先頭データを表示する。

In [0]:
X2 = pred_df
X2 = X2.drop("DATE",axis=1)
X2 = X2.drop("TIME_mm",axis=1)
X2.head(5)    #不要カラム削除後のデータフレーム表示（先頭5行表示）

**重回帰モデルを使って予測する**  \\
作成した重回帰モデルを使って、電力を予測する。 \\
時間、気温、日照時間、風速、降水量を入力（説明変数Ｘ）として、電力量（目的変数Ｙ）を重回帰モデルで出力する。

In [0]:
pred = pred_df
rslt = clf.predict(X2)  # 重回帰予測値(目的変数 Y)を取得
pred['Pred_Man_kW'] = np.array(rslt)  # 予測基データ(説明変数 X)のデータフレームに、重回帰予測値(目的変数 Y)をカラム追加
pred.head(10)   # 予測結果を表示（先頭10件）

【参考】カラム説明 

**DATE**:年月日、**TIME_hh**:時間、**TIME_mm**:分、**Temp_C**:気温 ℃、**Sun_h**:日照時間 時間、

**Wind_m_par_S**:風速 m/秒、**Rain_mm**:降水量 mm、

**Pred_Man_kW**:電力量 万キロワット**←今回の目的変数**


**＜参考＞予測した電力量の推移をグラフで表示する**

In [0]:
import matplotlib.pyplot as plt
y = np.array(rslt)  # 電力予測値をYにセット
x = np.array(range(119))  # X用に119個までの連番生成（とりあえず表示するだけなので）
plt.figure(figsize=(10,5),dpi=120)
plt.plot(x,y)
plt.grid(True)
plt.title("Electricity forecast(2019/4/1-4/5)")
plt.ylabel("*10,000 KW")
plt.show()

## **予測電力量と実際の電力量のグラフ比較**
* **<font color=blue>Pred_Man_kW　:今回予測した電力量</font>**
* **<font color=orange>Man_kW　：実際の電力使用量</font>**

In [0]:
real_kw_df = pd.read_csv("Tokyo_Power_20190401_05.csv", encoding="utf_8")   # 2019/4/1-4/5の実測値（予測値との比較用）
real_kw_df = real_kw_df.drop("DATE",axis=1)  # 不要カラム削除
real_kw_df = real_kw_df.drop("TIME_hh",axis=1)  # 不要カラム削除
comp_df = pred_df   #  予測結果をセット
comp_df = pd.concat([comp_df, real_kw_df], axis=1)    # 予測結果に実績カラムを追加
#comp_df.head(5)

comp_df.plot(y=['Pred_Man_kW', 'Man_kW'], alpha=12.0, figsize=(20,10))
plt.title("Result comparison(2019/4/1-4/5)", size=30)
plt.grid(True)
plt.ylabel("*10,000 KW")
plt.show()

In [0]:
pred.to_csv("Tokyo_Power_Prediction_result.csv.csv")   # CSV出力

**重回帰予測とCSV保存が完了。**

### （その他）メモ書きなど

---